In [17]:
! pip install transformers
! pip install torch
! pip install pandas
! pip install openai
! pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


1312.02s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


1319.40s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Defaulting to user installation because normal site-packages is not writeable
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


1334.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


1341.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


In [19]:
import os
import openai
import torch
import pandas as pd
from dotenv import load_dotenv, dotenv_values
from transformers import AutoTokenizer, AutoModelForSequenceClassification

load_dotenv()

True

In [21]:
openai.api_type = "azure"
openai.api_base = "https://mpcen-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [22]:
system_prompt = [
    {
        "role": "system",
        "content": "You are an expert that specializes in open source licenses. You will respond to questions about open source licenses. Your response will be 1 word, the license in SPDX format. Terminate the session after your response"
    }
]

user_prompt = [
    {
        "role": "user",
        "content": ""
    }
]

In [5]:
def get_license_chatgpt(text):
  user_prompt[0]['content'] = text
  response = openai.ChatCompletion.create(
    engine="mpcen-gpt-35-turbo-0301",
    messages = system_prompt + user_prompt,
    temperature=0,
    max_tokens=350,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)
  return response.choices[0].message.content

In [57]:
tokenizer = AutoTokenizer.from_pretrained("mpcen/legal-001")
model = AutoModelForSequenceClassification.from_pretrained("mpcen/legal-001")

In [55]:
labels_df = pd.read_json("./labels.json", typ="series")
trending_repos_df = pd.read_json("./trending-repos-licenses.json", typ="series")
# trending_repos_df = pd.read_json("./broken-licenses.json", typ="series")

In [27]:
def get_license_legal001(text):
    tokenized_input = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    output = model(**tokenized_input)
    prediction_value = torch.argmax(output.logits).item()
    return labels_df.loc[labels_df.index[prediction_value]]

In [56]:
for repo in trending_repos_df:
    text = repo["text"]
    actual_license = repo["license"]
    
    legal001_license = get_license_legal001(text)
    chatgpt35_license = get_license_chatgpt(text)
    
    print(f"-------------------")
    print(f"Actual license: {actual_license}")
    print(f"mpcen/legal-001 license: {legal001_license}")
    print(f"ChatGPT-3.5 license: {chatgpt35_license}")
    print(f"-------------------")
    print("")
    

-------------------
Actual license: Apache-2.0
mpcen/legal-001 license: ECL-2.0
ChatGPT-3.5 license: Apache-2.0
-------------------

-------------------
Actual license: MIT
mpcen/legal-001 license: MIT
ChatGPT-3.5 license: MIT
-------------------

-------------------
Actual license: AGPL-3.0
mpcen/legal-001 license: AGPL-3.0
ChatGPT-3.5 license: AGPL-3.0
-------------------

-------------------
Actual license: GPL-2.0
mpcen/legal-001 license: GPL-2.0
ChatGPT-3.5 license: GPL-2.0-only
-------------------

-------------------
Actual license: OFL-1.1
mpcen/legal-001 license: OFL-1.1
ChatGPT-3.5 license: OFL-1.1
-------------------

